<a href="https://colab.research.google.com/github/tirthpatel1111/Tops/blob/main/Nueral%20network%20and%20deep%20learning/Assignment/text_classifier_emotion_detaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle # creating .kaggle folder to store kaggle file
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d kushagra3204/sentiment-and-emotion-analysis-dataset

Dataset URL: https://www.kaggle.com/datasets/kushagra3204/sentiment-and-emotion-analysis-dataset
License(s): apache-2.0
  0% 0.00/14.9M [00:00<?, ?B/s]
100% 14.9M/14.9M [00:00<00:00, 1.61GB/s]


In [3]:
!unzip /content/sentiment-and-emotion-analysis-dataset.zip

Archive:  /content/sentiment-and-emotion-analysis-dataset.zip
  inflating: archive/combined_emotion.csv  
  inflating: archive/combined_sentiment_data.csv  


In [4]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 18.1 MB/s eta 0:00:00


In [5]:
import os
import numpy as np
import struct
import pickle
from array import array
%matplotlib inline
import random
import matplotlib.pyplot as plt
import gensim
from gensim.models import Word2Vec
from os.path  import join
import csv
import glob
import imageio
from glob import glob
import shutil
import json
from sklearn.impute import KNNImputer,SimpleImputer
import pandas as pd
import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset, Dataset
import torchvision.utils as vutils
from torchvision.io import read_image
from torchvision.transforms.functional import rgb_to_grayscale

import re
import math
import random

from collections import Counter


# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [6]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [8]:
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [9]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [10]:
df=pd.read_csv("/content/archive/combined_emotion.csv")

In [11]:
df.head(5)

,sentence,emotion
0,i just feel really helpless and heavy hearted,fear
1,ive enjoyed being able to slouch about relax a...,sad
2,i gave up my internship with the dmrg and am f...,fear
3,i dont know i feel so lost,sad
4,i am a kindergarten teacher and i am thoroughl...,fear


In [12]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

df['emotion']=le.fit_transform(df['emotion'])


In [13]:
df.sample(5)

,sentence,emotion
380389,i wanted no child would feel unloved or unsafe,4
236909,i see the time passing and my life standing st...,0
353670,i feel lethargic and i feel like i am dying al...,4
13199,i am thankful for my sweet little girl who kee...,2
418052,i ignored her minor tantrum and jumped down fr...,1


In [14]:
df['emotion'].value_counts()

,count
emotion,
2,143067
4,121187
0,59317
1,49649
3,34554
5,14972


In [15]:
def remove_punc(text):
  return re.sub(r'[^\w\s]','',text )

In [16]:
df["sentence"]=df["sentence"].apply(lambda x:remove_punc(x))

In [17]:
def get_wordnet_pos(treebank_tag):
    """Convert treebank POS tags to WordNet POS tags"""
    # if treebank_tag.startswith('J'):
    #     return wordnet.ADJ
    if treebank_tag.startswith('V'):
        return wordnet.VERB
    # elif treebank_tag.startswith('N'):
    #     return wordnet.NOUN
    # elif treebank_tag.startswith('R'):
    #     return wordnet.ADV
    else:
        return wordnet.NOUN  # default to noun

In [18]:
def lemmatize_text_nltk(text):
    """Lemmatize text with POS tagging"""
    if isinstance(text, float) or not text:
        return []

    # Tokenize and POS tag
    tokens = word_tokenize(text)
    # pos_tags = nltk.pos_tag(tokens)

    # # Lemmatize with POS information
    # lemmatizer = WordNetLemmatizer()
    # lemmas = []

    # for word, pos in pos_tags:
    #     wordnet_pos = get_wordnet_pos(pos)
    #     lemma = lemmatizer.lemmatize(word, pos=wordnet_pos)
    #     lemmas.append(lemma)

    return tokens

In [19]:
df['lemmatized_tokens'] = df['sentence'].apply(lemmatize_text_nltk)
print(df[['sentence', 'lemmatized_tokens']].head())
# df['sentence']=df['sentence'].apply(lambda x:word_tokenize(x))

                                            sentence  \
0      i just feel really helpless and heavy hearted   
1  ive enjoyed being able to slouch about relax a...   
2  i gave up my internship with the dmrg and am f...   
3                         i dont know i feel so lost   
4  i am a kindergarten teacher and i am thoroughl...   

                                   lemmatized_tokens  
0  [i, just, feel, really, helpless, and, heavy, ...  
1  [ive, enjoyed, being, able, to, slouch, about,...  
2  [i, gave, up, my, internship, with, the, dmrg,...  
3                 [i, dont, know, i, feel, so, lost]  
4  [i, am, a, kindergarten, teacher, and, i, am, ...  


In [20]:
df.shape

(422746, 3)

In [21]:
words = ['i', 'just', 'feel', 'really', 'helpless', 'and', 'heavy', 'hearted']
print(len(words))   # works → 8



8


In [22]:
df['token_count'] = df['lemmatized_tokens'].str.len()


In [23]:
df.drop('token_count',axis=1,inplace=True)

In [24]:
w2v_params = {
    "vector_size": 100,   # embedding size for Word2Vec
    "window": 10,
    "min_count": 2,
    "workers": 4,
    "epochs": 10,
    "seed": 42
}

In [25]:
w2v=Word2Vec(**w2v_params)

In [26]:
w2v.build_vocab(df['lemmatized_tokens'])

In [27]:
w2v.train(df['lemmatized_tokens'],total_examples=w2v.corpus_count,epochs=w2v.epochs)

(55518461, 81252540)

In [28]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df['lemmatized_tokens'],df['emotion'],test_size=0.2,random_state=42)

In [29]:
counter=Counter()
for s in df['lemmatized_tokens']:
  counter.update(s)

In [30]:
counter

Counter({'i': 685827,
         'just': 37473,
         'feel': 293986,
         'really': 26185,
         'helpless': 2987,
         'and': 253802,
         'heavy': 418,
         'hearted': 129,
         'ive': 15065,
         'enjoyed': 622,
         'being': 18484,
         'able': 4896,
         'to': 236530,
         'slouch': 2,
         'about': 47021,
         'relax': 405,
         'unwind': 35,
         'frankly': 237,
         'needed': 1433,
         'it': 85627,
         'after': 8375,
         'those': 7349,
         'last': 5726,
         'few': 5152,
         'weeks': 2112,
         'around': 7758,
         'the': 219819,
         'end': 4310,
         'of': 131803,
         'uni': 123,
         'expo': 6,
         'have': 62102,
         'lately': 2991,
         'started': 5092,
         'find': 6009,
         'myself': 21972,
         'feeling': 136149,
         'a': 165233,
         'bit': 13083,
         'listless': 891,
         'which': 9782,
         'is': 57843,

In [31]:
len(counter)

75297

In [32]:
min_frq=0
PAD,UNK="<pad>","<unk>"
max_vocab=len(counter)+2
itos=[PAD,UNK]+[tok for tok,c in counter.items() if c>=min_frq]
stoi={tok:i for i,tok in enumerate(itos)}
VOCAB_SIZE=len(itos)
print("vocab size",VOCAB_SIZE)
print("itos",itos)
print("stoi",stoi)

vocab size 75299
itos ['<pad>', '<unk>', 'i', 'just', 'feel', 'really', 'helpless', 'and', 'heavy', 'hearted', 'ive', 'enjoyed', 'being', 'able', 'to', 'slouch', 'about', 'relax', 'unwind', 'frankly', 'needed', 'it', 'after', 'those', 'last', 'few', 'weeks', 'around', 'the', 'end', 'of', 'uni', 'expo', 'have', 'lately', 'started', 'find', 'myself', 'feeling', 'a', 'bit', 'listless', 'which', 'is', 'never', 'good', 'thing', 'gave', 'up', 'my', 'internship', 'with', 'dmrg', 'am', 'distraught', 'dont', 'know', 'so', 'lost', 'kindergarten', 'teacher', 'thoroughly', 'weary', 'job', 'having', 'taken', 'university', 'entrance', 'exam', 'suffered', 'from', 'anxiety', 'for', 'as', 'did', 'not', 'want', 'carry', 'on', 'work', 'studies', 'were', 'only', 'alternative', 'was', 'beginning', 'quite', 'disheartened', 'would', 'think', 'that', 'whomever', 'be', 'lucky', 'enough', 'stay', 'in', 'this', 'suite', 'must', 'like', 'most', 'romantic', 'place', 'earth', 'fear', 'they', 'won', 't', 'ever', 'de

In [33]:
emb_dim = w2v.vector_size
embedding_matrix = np.random.normal(scale=0.6, size=(VOCAB_SIZE, emb_dim)).astype(np.float32)
for tok, idx in stoi.items():
    if tok in w2v.wv:
        embedding_matrix[idx] = w2v.wv[tok]
embedding_matrix[stoi[PAD]] = np.zeros(emb_dim, dtype=np.float32)
embedding_tensor = torch.tensor(embedding_matrix)

In [34]:
len(itos)

75299

In [35]:
MAX_LEN=178
def encode(toks, max_len=MAX_LEN):
    # toks = tokenize(text)
    idxs = [stoi.get(t, stoi[UNK]) for t in toks][:max_len]
    if len(idxs) < max_len:
        idxs += [stoi[PAD]]*(max_len - len(idxs))
    return np.array(idxs, dtype=np.int64)

In [36]:
encode(df['lemmatized_tokens'][5])

array([ 2, 84, 85, 14,  4, 86, 87,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0])

In [37]:
class ReviewDataset(Dataset):
    def __init__(self, feathers,labels):        # force them into plain Python lists
        self.texts = feathers.tolist()
        self.labels = labels.astype(int).tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, i):
        return (
            torch.tensor(encode(self.texts[i]), dtype=torch.long),
            torch.tensor(self.labels[i], dtype=torch.long)
        )

In [38]:
train_dataset=ReviewDataset(x_train,y_train)
test_dataset=ReviewDataset(x_test,y_test)

In [39]:
x_train.head()

,lemmatized_tokens
189081,"[i, feel, as, though, he, convinced, me, that,..."
235833,"[i, feel, i, have, a, lot, of, valuable, infor..."
313065,"[id, better, say, something, nice, for, the, s..."
156251,"[my, sister, and, brother, in, law, returned, ..."
325534,"[i, feel, valued, and, somewhat, smug]"


In [40]:
train_loader=DataLoader(train_dataset,batch_size=64,shuffle=False)
test_loader=DataLoader(test_dataset,batch_size=64,shuffle=True)

In [41]:
def decode(num):
  return [itos[i] for i in num]

In [42]:
print(next(iter(train_loader)))

[tensor([[  2,   4,  73,  ...,   0,   0,   0],
        [  2,   4,   2,  ...,   0,   0,   0],
        [508, 192, 840,  ...,   0,   0,   0],
        ...,
        [  2, 130, 108,  ...,   0,   0,   0],
        [  2,   5, 100,  ...,   0,   0,   0],
        [  2, 902,   4,  ...,   0,   0,   0]]), tensor([2, 2, 2, 2, 2, 2, 0, 2, 4, 0, 2, 2, 4, 3, 3, 4, 2, 5, 2, 2, 2, 4, 2, 4,
        4, 1, 5, 2, 2, 3, 2, 2, 1, 3, 4, 0, 2, 4, 4, 4, 5, 4, 2, 2, 2, 1, 2, 4,
        3, 2, 1, 3, 2, 4, 0, 4, 2, 0, 0, 2, 5, 1, 3, 0])]


In [43]:
decode([    1,     1,     2,     1,     1,     1,     1,  3458,  3176,  3176,
          3501,     1,     1,     1,     1,    39,   349,     1,     1,     1,
             1,   108,  5082,  4558,  1256,  6465,  5082,     1,     1,     1,
             1,  5082,  3176,     1,     1,     1,     1,  2707,  4558,  2245,
         12396,     2,  2245,  2707,  3176,   457,     1,     1,     1,     1,
           337,  3176,     1,     1,     1,     1,   108,  5082,    39,   108,
             1,     1,     1,     1,     2,     1,     1,     1,     1,   390,
            39,   349,     1,     1,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0])

['<unk>',
 '<unk>',
 'i',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 'f',
 'e',
 'e',
 'l',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 'a',
 's',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 't',
 'h',
 'o',
 'u',
 'g',
 'h',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 'h',
 'e',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 'c',
 'o',
 'n',
 'v',
 'i',
 'n',
 'c',
 'e',
 'd',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 'm',
 'e',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 't',
 'h',
 'a',
 't',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 'i',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 'w',
 'a',
 's',
 '<unk>',
 '<unk>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad

In [44]:
trian_lodaer=iter(train_loader)
data,label=next(trian_lodaer)
print(data)
print(label)

tensor([[  2,   4,  73,  ...,   0,   0,   0],
        [  2,   4,   2,  ...,   0,   0,   0],
        [508, 192, 840,  ...,   0,   0,   0],
        ...,
        [  2, 130, 108,  ...,   0,   0,   0],
        [  2,   5, 100,  ...,   0,   0,   0],
        [  2, 902,   4,  ...,   0,   0,   0]])
tensor([2, 2, 2, 2, 2, 2, 0, 2, 4, 0, 2, 2, 4, 3, 3, 4, 2, 5, 2, 2, 2, 4, 2, 4,
        4, 1, 5, 2, 2, 3, 2, 2, 1, 3, 4, 0, 2, 4, 4, 4, 5, 4, 2, 2, 2, 1, 2, 4,
        3, 2, 1, 3, 2, 4, 0, 4, 2, 0, 0, 2, 5, 1, 3, 0])


In [45]:
stoi.get('feel',stoi[UNK])

4

In [46]:
encode(df['lemmatized_tokens'][1])

array([10, 11, 12, 13, 14, 15, 16, 17,  7, 18,  7, 19, 20, 21, 22, 23, 24,
       25, 26, 27, 28, 29, 30, 31,  7, 28, 32,  2, 33, 34, 35, 14, 36, 37,
       38, 39, 40, 41, 42, 43, 44,  5, 39, 45, 46,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0])

In [47]:
df['lemmatized_tokens'][0]

['i', 'just', 'feel', 'really', 'helpless', 'and', 'heavy', 'hearted']

In [48]:
class BiderctionalLstm(nn.Module):
   def __init__(self, vocab_size, emb_dim=100, dropout=0.2, num_classes=6):
      super().__init__()
      self.emb=nn.Embedding(VOCAB_SIZE,emb_dim,padding_idx=stoi[PAD])

      self.emb.weight.data.copy_(embedding_tensor)
      self.emb.weight.requires_grad = False
      self.lstm1=nn.LSTM(emb_dim,128,batch_first=True,bidirectional=True)
      self.lstm2=nn.LSTM(128*2,64,batch_first=True,bidirectional=True)
      self.lstm3=nn.LSTM(64*2,32,batch_first=True,bidirectional=True)
      self.network=nn.Sequential(
          nn.Dropout(0.3),
          nn.Linear(32*2,64),
          nn.BatchNorm1d(64),
          nn.ReLU(),

          nn.Dropout(0.1),
          nn.Linear(64,128),
          nn.BatchNorm1d(128),
          nn.ReLU(),

          nn.Dropout(0.1),
          nn.Linear(128,64),
          nn.BatchNorm1d(64),
          nn.ReLU(),

          nn.Linear(64,num_classes)

      )
   def forward(self,x):
      x=self.emb(x)

      x,_=self.lstm1(x)
      x,_=self.lstm2(x)
      x,(hidden, cell)=self.lstm3(x)
      hidden_cat=torch.cat([hidden[0],hidden[1]],dim=1)
      out=self.network(hidden_cat)
      return out


In [49]:
def train_per_epoch(model,loader,criterion,optimizer,device):
  model.train()
  total_loss=0.0
  for data,label in loader:
    data,label=data.to(device),label.to(device)
    optimizer.zero_grad()
    output=model(data)
    loss=criterion(output,label)
    loss.backward()
    optimizer.step()
    total_loss+=loss.item()*data.size(0)
  avg_loss=total_loss/len(loader.dataset)
  return avg_loss

In [50]:
from sklearn.metrics import accuracy_score, f1_score


def eval_model(model, loader, device):

    model.eval()
    all_y, all_p = [], []
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device).long()


        preds = model(xb)
        pred_labels = preds.argmax(dim=1)

        all_y.extend(yb.cpu().numpy().tolist())
        all_p.extend(pred_labels.cpu().numpy().tolist())

    y, p = np.array(all_y), np.array(all_p)
    return {
        "accuracy": accuracy_score(y, p),
        "f1-score": f1_score(y, p, average="weighted"),  # weighted for multi-class
    }


In [51]:
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

In [52]:
def run_experiment(model_class,name,epoch=25,lr=0.01,train_loader=train_loader,test_loader=test_loader):
  print(f"Running experiment: {name}")
  model=model_class(VOCAB_SIZE).to(DEVICE)
  crit = nn.CrossEntropyLoss()
  opt = torch.optim.AdamW(model.parameters(), lr=lr)
  for e in range(1,epoch+1):
    train_loss=train_per_epoch(model,train_loader,crit,opt,DEVICE)
    val_metrics=eval_model(model,test_loader,DEVICE)
    print(f"Epoch {e:02d} | loss={train_loss:.4f} | val_accuracy={val_metrics['accuracy']:.4f} | val_f1={val_metrics['f1-score']:.4f} ")


In [53]:
results = []
for model_class, name in [(BiderctionalLstm,"LSTM")]:
    results.append(run_experiment(model_class, name))

Running experiment: LSTM
Epoch 01 | loss=0.2972 | val_accuracy=0.9299 | val_f1=0.9295 
Epoch 02 | loss=0.1491 | val_accuracy=0.9349 | val_f1=0.9329 
Epoch 03 | loss=0.1376 | val_accuracy=0.9360 | val_f1=0.9349 
Epoch 04 | loss=0.1286 | val_accuracy=0.9253 | val_f1=0.9213 
Epoch 05 | loss=0.1256 | val_accuracy=0.9375 | val_f1=0.9352 
Epoch 06 | loss=0.1268 | val_accuracy=0.9346 | val_f1=0.9323 
Epoch 07 | loss=0.1242 | val_accuracy=0.9290 | val_f1=0.9263 
Epoch 08 | loss=0.1238 | val_accuracy=0.9334 | val_f1=0.9311 
Epoch 09 | loss=0.1198 | val_accuracy=0.9380 | val_f1=0.9359 
Epoch 10 | loss=0.1242 | val_accuracy=0.9387 | val_f1=0.9366 
Epoch 11 | loss=0.1214 | val_accuracy=0.9375 | val_f1=0.9351 
Epoch 12 | loss=0.1212 | val_accuracy=0.9384 | val_f1=0.9362 
Epoch 13 | loss=0.1204 | val_accuracy=0.9376 | val_f1=0.9357 
Epoch 14 | loss=0.1220 | val_accuracy=0.9384 | val_f1=0.9363 
Epoch 15 | loss=0.1188 | val_accuracy=0.9366 | val_f1=0.9344 
Epoch 16 | loss=0.1225 | val_accuracy=0.9324 

KeyboardInterrupt: 